In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import pickle
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from ipyparallel import Client
from Bio import SeqIO
import pyfaidx
import pyfasta
import textwrap
import tempfile
%matplotlib inline
from collections import Counter

In [0]:
cd "~/eckertlab/Mitra/mapping/split_parallel/collapsed/"

In [0]:
mapped = !ls *mapped.bam

In [0]:
samtools = "/home/cfriedline/bin/samtools"

In [0]:
from subprocess import Popen, PIPE, STDOUT

In [0]:
rc = Client(profile = "sge")
dv = rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
with dv.sync_imports():
    from subprocess import Popen, PIPE, STDOUT
    import os
    import socket

In [0]:
def get_contigs(bam):
    contigs = set()
    cmd = "samtools view %s" % bam
    p = Popen(cmd, stdout=PIPE, shell=True)
    for line in p.stdout:
        d = line.decode().split("\t")
        contigs.add(d[2])
    return contigs

In [0]:
dv.push({'samtools': samtools,
        'get_contigs': get_contigs}).r;

In [0]:
jobs = []
for b in mapped:
    jobs.append(lv.apply_async(get_contigs, b))

In [0]:
np.sum([x.ready() for x in jobs])

In [0]:
all_contigs = set()
for j in jobs:
    for contig in j.r:
        all_contigs.add(contig)

In [0]:
contig_counts = Counter()
for j in jobs:
    for contig in j.r:
        contig_counts[contig] += 1

In [0]:
all_contigs_sav = "all_contigs.pkl"

In [0]:
contig_counts_sav = "contig_counts.pkl"

In [0]:
pickle.dump(all_contigs, open(all_contigs_sav, "wb"), pickle.HIGHEST_PROTOCOL)

In [0]:
pickle.dump(contig_counts, open(contig_counts_sav, "wb"), pickle.HIGHEST_PROTOCOL)

In [0]:
all_contigs = pickle.load(open(all_contigs_sav, "rb"))

In [0]:
contig_counts = pickle.load(open(contig_counts_sav, "rb"))

In [0]:
assembly = "/home/cfriedline/eckertlab/SugarPine_genome/pila.v1.0.scafSeq"

In [0]:
#count = !grep -c ">" $assembly

In [0]:
count = 58407655 #save for later

In [0]:
len(all_contigs)

In [0]:
len(all_contigs), len(all_contigs)/count

In [0]:
mapped_fasta = "%s_mapped.fasta" % assembly

### in external session, to generate the index
```
f = pyfasta.Fasta(assembly)
```

In [0]:
f = pyfasta.Fasta(assembly)

In [0]:
c, f[c], len(f[c][:])

In [0]:
def fill(text, width=70):
    return '\n'.join(text[i:i+width] for i in
                     range(0, len(text), width))

def write_mapped_fasta():
    with open(mapped_fasta, "w", buffering=10240) as o:
        for i, c in enumerate(all_contigs):
            o.write(">%s\n%s\n" % (c, fill(f[c][:], width=100)))
            if i % 1000 == 0:
                print("at %d" % i)

In [0]:
%connect_info

### Index mapped file
```python
m = pyfasta.Fasta(mapped_fasta)
```

```bash
samtools faidx /home/cfriedline/eckertlab/SugarPine_genome/pila.v1.0.scafSeq_mapped.fasta
```

In [0]:
mapped_fasta

In [0]:
m = pyfasta.Fasta(mapped_fasta)

In [0]:
len(m)

In [0]:
contig_data = {}

In [0]:
for seq in contig_counts:
    if not seq in contig_data:
        contig_data[seq] = {}
    contig_data[seq]['length'] = len(m[seq])
    contig_data[seq]['hits'] = contig_counts[seq]

In [0]:
with open("contig_length.txt", "w") as o:
    for seq in contig_counts:
        o.write("%s\t%d\n" % (seq, contig_data[seq]['length']))

In [0]:
with open("contigs.bed", "w") as o:
    for seq in contig_counts:
        o.write("%s\t%d\t%d\n" % (seq, 0, contig_data[seq]['length']))

In [0]:
contig_df = pd.DataFrame(contig_data).T

In [0]:
contig_df.describe()

In [0]:
plt.scatter(contig_df['length'], contig_df['hits']);
plt.xlabel("contig length")
plt.ylabel("# hits")
# plt.ylim(0, 5000000)
plt.show()

In [0]:
lens = pd.Series(lens)

In [0]:
sns.distplot(lens, kde=False, bins=500)
plt.xlim(0,500000)
plt.show()

In [0]:
lens.describe()